# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 12 2022 8:30AM,248402,10,MSP214785,Methapharm-G,Released
1,Oct 12 2022 8:30AM,248403,10,MSP214742,"Methapharm, Inc.",Released
2,Oct 12 2022 8:30AM,248404,10,0086070474,ISDIN Corporation,Released
3,Oct 12 2022 8:30AM,248404,10,0086070475,ISDIN Corporation,Released
4,Oct 12 2022 8:30AM,248404,10,0086070493,ISDIN Corporation,Released
5,Oct 12 2022 8:30AM,248404,10,0086070498,ISDIN Corporation,Released
6,Oct 12 2022 8:30AM,248404,10,0086070509,ISDIN Corporation,Released
7,Oct 12 2022 8:30AM,248404,10,0086070515,ISDIN Corporation,Released
8,Oct 12 2022 8:30AM,248404,10,0086070514,ISDIN Corporation,Released
9,Oct 12 2022 8:30AM,248404,10,0086070519,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,248400,Released,1
24,248402,Released,1
25,248403,Released,1
26,248404,Released,11
27,248405,Released,23


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248400,NaN,NaN,1.0
248402,NaN,NaN,1.0
248403,NaN,NaN,1.0
248404,NaN,NaN,11.0
248405,NaN,NaN,23.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248038,0.0,1.0,0.0
248080,0.0,0.0,1.0
248316,0.0,0.0,1.0
248317,0.0,0.0,1.0
248344,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248038,0,1,0
248080,0,0,1
248316,0,0,1
248317,0,0,1
248344,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248038,0,1,0
1,248080,0,0,1
2,248316,0,0,1
3,248317,0,0,1
4,248344,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248038,,1,
1,248080,,,1
2,248316,,,1
3,248317,,,1
4,248344,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 12 2022 8:30AM,248402,10,Methapharm-G
1,Oct 12 2022 8:30AM,248403,10,"Methapharm, Inc."
2,Oct 12 2022 8:30AM,248404,10,ISDIN Corporation
13,Oct 12 2022 8:30AM,248405,10,ISDIN Corporation
36,Oct 12 2022 8:30AM,248396,10,ISDIN Corporation
37,Oct 11 2022 4:13PM,248400,19,"GCH Granules USA, Inc."
38,Oct 11 2022 3:49PM,248399,21,"NBTY Global, Inc."
39,Oct 11 2022 3:48PM,248398,21,"NBTY Global, Inc."
40,Oct 11 2022 3:48PM,248397,21,"NBTY Global, Inc."
41,Oct 11 2022 3:11PM,248390,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 12 2022 8:30AM,248402,10,Methapharm-G,,,1
1,Oct 12 2022 8:30AM,248403,10,"Methapharm, Inc.",,,1
2,Oct 12 2022 8:30AM,248404,10,ISDIN Corporation,,,11
3,Oct 12 2022 8:30AM,248405,10,ISDIN Corporation,,,23
4,Oct 12 2022 8:30AM,248396,10,ISDIN Corporation,,,1
5,Oct 11 2022 4:13PM,248400,19,"GCH Granules USA, Inc.",,,1
6,Oct 11 2022 3:49PM,248399,21,"NBTY Global, Inc.",,,1
7,Oct 11 2022 3:48PM,248398,21,"NBTY Global, Inc.",,,1
8,Oct 11 2022 3:48PM,248397,21,"NBTY Global, Inc.",,,1
9,Oct 11 2022 3:11PM,248390,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 12 2022 8:30AM,248402,10,Methapharm-G,1,,
1,Oct 12 2022 8:30AM,248403,10,"Methapharm, Inc.",1,,
2,Oct 12 2022 8:30AM,248404,10,ISDIN Corporation,11,,
3,Oct 12 2022 8:30AM,248405,10,ISDIN Corporation,23,,
4,Oct 12 2022 8:30AM,248396,10,ISDIN Corporation,1,,
5,Oct 11 2022 4:13PM,248400,19,"GCH Granules USA, Inc.",1,,
6,Oct 11 2022 3:49PM,248399,21,"NBTY Global, Inc.",1,,
7,Oct 11 2022 3:48PM,248398,21,"NBTY Global, Inc.",1,,
8,Oct 11 2022 3:48PM,248397,21,"NBTY Global, Inc.",1,,
9,Oct 11 2022 3:11PM,248390,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 12 2022 8:30AM,248402,10,Methapharm-G,1,,
1,Oct 12 2022 8:30AM,248403,10,"Methapharm, Inc.",1,,
2,Oct 12 2022 8:30AM,248404,10,ISDIN Corporation,11,,
3,Oct 12 2022 8:30AM,248405,10,ISDIN Corporation,23,,
4,Oct 12 2022 8:30AM,248396,10,ISDIN Corporation,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 12 2022 8:30AM,248402,10,Methapharm-G,1.0,NaN,NaN
1,Oct 12 2022 8:30AM,248403,10,"Methapharm, Inc.",1.0,NaN,NaN
2,Oct 12 2022 8:30AM,248404,10,ISDIN Corporation,11.0,NaN,NaN
3,Oct 12 2022 8:30AM,248405,10,ISDIN Corporation,23.0,NaN,NaN
4,Oct 12 2022 8:30AM,248396,10,ISDIN Corporation,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 12 2022 8:30AM,248402,10,Methapharm-G,1.0,0.0,0.0
1,Oct 12 2022 8:30AM,248403,10,"Methapharm, Inc.",1.0,0.0,0.0
2,Oct 12 2022 8:30AM,248404,10,ISDIN Corporation,11.0,0.0,0.0
3,Oct 12 2022 8:30AM,248405,10,ISDIN Corporation,23.0,0.0,0.0
4,Oct 12 2022 8:30AM,248396,10,ISDIN Corporation,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1738765,64.0,0.0,5.0
12,496755,0.0,2.0,0.0
16,745007,2.0,1.0,0.0
18,248317,1.0,0.0,0.0
19,745113,8.0,3.0,21.0
21,1489699,5.0,1.0,0.0
22,745167,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1738765,64.0,0.0,5.0
1,12,496755,0.0,2.0,0.0
2,16,745007,2.0,1.0,0.0
3,18,248317,1.0,0.0,0.0
4,19,745113,8.0,3.0,21.0
5,21,1489699,5.0,1.0,0.0
6,22,745167,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,64.0,0.0,5.0
1,12,0.0,2.0,0.0
2,16,2.0,1.0,0.0
3,18,1.0,0.0,0.0
4,19,8.0,3.0,21.0
5,21,5.0,1.0,0.0
6,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,64.0
1,12,Released,0.0
2,16,Released,2.0
3,18,Released,1.0
4,19,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,21,22
Status,,,,,,,
Completed,5.0,0.0,0.0,0.0,21.0,0.0,0.0
Executing,0.0,2.0,1.0,0.0,3.0,1.0,0.0
Released,64.0,0.0,2.0,1.0,8.0,5.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,21,22
0,Completed,5.0,0.0,0.0,0.0,21.0,0.0,0.0
1,Executing,0.0,2.0,1.0,0.0,3.0,1.0,0.0
2,Released,64.0,0.0,2.0,1.0,8.0,5.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,21,22
0,Completed,5.0,0.0,0.0,0.0,21.0,0.0,0.0
1,Executing,0.0,2.0,1.0,0.0,3.0,1.0,0.0
2,Released,64.0,0.0,2.0,1.0,8.0,5.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()